In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import os
from os import listdir
import readInDataFiles

In [36]:
MocapDFs = readInDataFiles.getMocapData(r"/Users/jacobvogel/Desktop/Blister Labs/GitHub/Blister/Wheel_Deflection/0_Data/Dynamic Radial/Stans Flow 6-26-23 Tire On/Optitrack Data")

MocapDFs[2].describe()

,Frame,Time (sec),Unlabeled 1585 X,Unlabeled 1585 Y,Unlabeled 1585 Z,Unlabeled 1586 X,Unlabeled 1586 Y,Unlabeled 1586 Z,Unlabeled 1587 X,Unlabeled 1587 Y,Unlabeled 1587 Z
count,9033,9033,9033,9033,9033,9033,9033,9033,9033,9033,9033
unique,9033,9033,5477,2766,2317,4890,1712,1681,4492,6832,1936
top,0,0.000000,28.236906,-101.122063,157.485016,23.772881,266.866699,281.190857,35.226902,10.171844,286.878601
freq,1,1,10,28,40,12,94,82,12,7,55


In [37]:
toBeCleanedTesting = MocapDFs[0]
toBeCleanedTesting.describe()

,Frame,Time (sec),Unlabeled 1557 X,Unlabeled 1557 Y,Unlabeled 1557 Z,Unlabeled 1558 X,Unlabeled 1558 Y,Unlabeled 1558 Z,Unlabeled 1559 X,Unlabeled 1559 Y,Unlabeled 1559 Z
count,4599,4599,4599,4599,4599,4599,4599,4599,4599,4599,4599
unique,4599,4599,3751,2475,2181,3704,1734,1848,3411,4292,1727
top,0,0.000000,28.113628,-101.130325,157.579102,24.131794,267.084106,280.995056,35.103909,10.326365,286.850616
freq,1,1,6,10,19,6,34,27,7,4,20


In [38]:
toBeCleanedTesting.head()

,Frame,Time (sec),Unlabeled 1557 X,Unlabeled 1557 Y,Unlabeled 1557 Z,Unlabeled 1558 X,Unlabeled 1558 Y,Unlabeled 1558 Z,Unlabeled 1559 X,Unlabeled 1559 Y,Unlabeled 1559 Z
3,0,0.000000,28.113848,-101.129868,157.576630,24.130949,267.084473,280.998383,35.100338,10.319546,286.849518
4,1,0.002000,28.113277,-101.132439,157.580048,24.135048,267.085358,280.998962,35.100655,10.322523,286.852386
5,2,0.004000,28.112267,-101.131912,157.580322,24.131615,267.085327,280.996948,35.102345,10.323091,286.847656
6,3,0.006000,28.110779,-101.131294,157.579559,24.129690,267.083282,281.000214,35.099350,10.321952,286.851776
7,4,0.008000,28.116693,-101.133499,157.580215,24.131927,267.083282,280.996826,35.103989,10.320887,286.853821


In [39]:
#Getting all of the y values of each marker and finding the max
firstRow = toBeCleanedTesting.iloc[0]
YVals = firstRow[3::3]

maxY = max(YVals)

#findingRimTop = YVals
YVals

Unlabeled 1557 Y    -101.129868
Unlabeled 1558 Y     267.084473
Unlabeled 1559 Y      10.319546
Name: 3, dtype: object

In [40]:
#Labeling the columns corresponding to the rim top marker
for index in YVals.index:
    if YVals[index] == maxY:
        # Rim Top Y Column -> rimTopY
        num_index = toBeCleanedTesting.columns.get_loc(index)
        toBeCleanedTesting.rename(columns={toBeCleanedTesting.columns[num_index]: 'rim_top_y'}, inplace=True)
        toBeCleanedTesting.rename(columns={toBeCleanedTesting.columns[num_index-1]: 'rim_top_x'}, inplace=True)
        toBeCleanedTesting.rename(columns={toBeCleanedTesting.columns[num_index+1]: 'rim_top_z'}, inplace=True)
        YVals = YVals.drop(index)

toBeCleanedTesting.head()

,Frame,Time (sec),Unlabeled 1557 X,Unlabeled 1557 Y,Unlabeled 1557 Z,rim_top_x,rim_top_y,rim_top_z,Unlabeled 1559 X,Unlabeled 1559 Y,Unlabeled 1559 Z
3,0,0.000000,28.113848,-101.129868,157.576630,24.130949,267.084473,280.998383,35.100338,10.319546,286.849518
4,1,0.002000,28.113277,-101.132439,157.580048,24.135048,267.085358,280.998962,35.100655,10.322523,286.852386
5,2,0.004000,28.112267,-101.131912,157.580322,24.131615,267.085327,280.996948,35.102345,10.323091,286.847656
6,3,0.006000,28.110779,-101.131294,157.579559,24.129690,267.083282,281.000214,35.099350,10.321952,286.851776
7,4,0.008000,28.116693,-101.133499,157.580215,24.131927,267.083282,280.996826,35.103989,10.320887,286.853821


In [41]:
toBeCleanedTesting.columns[7]

'rim_top_z'

In [42]:
#recalculate max to find axel column
maxY = max(YVals)

for index in YVals.index:
    if YVals[index] == maxY:
        # Axel Top Y Column -> axelY
        num_index = toBeCleanedTesting.columns.get_loc(index)
        toBeCleanedTesting.rename(columns={toBeCleanedTesting.columns[num_index]: 'axel_y'}, inplace=True)
        toBeCleanedTesting.rename(columns={toBeCleanedTesting.columns[num_index-1]: 'axel_x'}, inplace=True)
        toBeCleanedTesting.rename(columns={toBeCleanedTesting.columns[num_index+1]: 'axel_z'}, inplace=True)
        YVals = YVals.drop(index)
        
toBeCleanedTesting.head()

,Frame,Time (sec),Unlabeled 1557 X,Unlabeled 1557 Y,Unlabeled 1557 Z,rim_top_x,rim_top_y,rim_top_z,axel_x,axel_y,axel_z
3,0,0.000000,28.113848,-101.129868,157.576630,24.130949,267.084473,280.998383,35.100338,10.319546,286.849518
4,1,0.002000,28.113277,-101.132439,157.580048,24.135048,267.085358,280.998962,35.100655,10.322523,286.852386
5,2,0.004000,28.112267,-101.131912,157.580322,24.131615,267.085327,280.996948,35.102345,10.323091,286.847656
6,3,0.006000,28.110779,-101.131294,157.579559,24.129690,267.083282,281.000214,35.099350,10.321952,286.851776
7,4,0.008000,28.116693,-101.133499,157.580215,24.131927,267.083282,280.996826,35.103989,10.320887,286.853821


In [43]:
YVals

Unlabeled 1557 Y    -101.129868
Name: 3, dtype: object

In [46]:
YVals.index[0]

'Unlabeled 1557 Y'

In [47]:
#Re-labelling the xyz columns of the stand marker
num_index = toBeCleanedTesting.columns.get_loc(YVals.index[0])
toBeCleanedTesting.rename(columns={toBeCleanedTesting.columns[num_index]: 'stand_y'}, inplace=True)
toBeCleanedTesting.rename(columns={toBeCleanedTesting.columns[num_index-1]: 'stand_x'}, inplace=True)
toBeCleanedTesting.rename(columns={toBeCleanedTesting.columns[num_index+1]: 'stand_z'}, inplace=True)

toBeCleanedTesting.head()

,Frame,Time (sec),stand_x,stand_y,stand_z,rim_top_x,rim_top_y,rim_top_z,axel_x,axel_y,axel_z
3,0,0.000000,28.113848,-101.129868,157.576630,24.130949,267.084473,280.998383,35.100338,10.319546,286.849518
4,1,0.002000,28.113277,-101.132439,157.580048,24.135048,267.085358,280.998962,35.100655,10.322523,286.852386
5,2,0.004000,28.112267,-101.131912,157.580322,24.131615,267.085327,280.996948,35.102345,10.323091,286.847656
6,3,0.006000,28.110779,-101.131294,157.579559,24.129690,267.083282,281.000214,35.099350,10.321952,286.851776
7,4,0.008000,28.116693,-101.133499,157.580215,24.131927,267.083282,280.996826,35.103989,10.320887,286.853821


In [62]:
#finds original distance between the stand and rim markers

firstRow = toBeCleanedTesting.iloc[0]

distRimStand0 = {}

distRimStandx = float(firstRow['rim_top_x'])-float(firstRow['stand_x'])
distRimStand0["x"] = distRimStandx

distRimStandy = float(firstRow['rim_top_y'])-float(firstRow['stand_y'])
distRimStand0["y"] = distRimStandy

distRimStandz = float(firstRow['rim_top_z'])-float(firstRow['stand_z'])
distRimStand0["z"] = distRimStandz
distRimStand0



{'x': -3.9828989999999997, 'y': 368.214341, 'z': 123.421753}

In [68]:
"""
first finds the distance between the stand and rim for each point in time
and subtracts the original distance to filter out noise from the stand
moving due to the carrige and piston catch
"""

dataLength = len(toBeCleanedTesting)
distRimStand = {}

for i in range(0, dataLength):
    row = toBeCleanedTesting.iloc[i]
    distRimStand


Frame                   0
Time (sec)       0.000000
stand_x         28.113848
stand_y       -101.129868
stand_z        157.576630
rim_top_x       24.130949
rim_top_y      267.084473
rim_top_z      280.998383
axel_x          35.100338
axel_y          10.319546
axel_z         286.849518
Name: 3, dtype: object
Frame                   1
Time (sec)       0.002000
stand_x         28.113277
stand_y       -101.132439
stand_z        157.580048
rim_top_x       24.135048
rim_top_y      267.085358
rim_top_z      280.998962
axel_x          35.100655
axel_y          10.322523
axel_z         286.852386
Name: 4, dtype: object
Frame                   2
Time (sec)       0.004000
stand_x         28.112267
stand_y       -101.131912
stand_z        157.580322
rim_top_x       24.131615
rim_top_y      267.085327
rim_top_z      280.996948
axel_x          35.102345
axel_y          10.323091
axel_z         286.847656
Name: 5, dtype: object
Frame                   3
Time (sec)       0.006000
stand_x         28.11

Frame                 963
Time (sec)       1.926000
stand_x         28.117161
stand_y       -101.138168
stand_z        157.577682
rim_top_x       24.128468
rim_top_y      267.078064
rim_top_z      280.997589
axel_x          35.103233
axel_y          10.317164
axel_z         286.847839
Name: 966, dtype: object
Frame                 964
Time (sec)       1.928000
stand_x         28.114189
stand_y       -101.135056
stand_z        157.580383
rim_top_x       24.129389
rim_top_y      267.082825
rim_top_z      280.998047
axel_x          35.100273
axel_y          10.316476
axel_z         286.851135
Name: 967, dtype: object
Frame                 965
Time (sec)       1.930000
stand_x         28.113943
stand_y       -101.134041
stand_z        157.577682
rim_top_x       24.132072
rim_top_y      267.082733
rim_top_z      280.996704
axel_x          35.102932
axel_y          10.325529
axel_z         286.847717
Name: 968, dtype: object
Frame                 966
Time (sec)       1.932000
stand_x        

Frame                1743
Time (sec)       3.486000
stand_x         28.117462
stand_y       -101.131752
stand_z        157.578995
rim_top_x       24.137295
rim_top_y      267.081573
rim_top_z      280.996063
axel_x          35.108486
axel_y          10.322099
axel_z         286.854614
Name: 1746, dtype: object
Frame                1744
Time (sec)       3.488000
stand_x         28.115829
stand_y       -101.133080
stand_z        157.578232
rim_top_x       24.136106
rim_top_y      267.085297
rim_top_z      280.995270
axel_x          35.104729
axel_y          10.326465
axel_z         286.847961
Name: 1747, dtype: object
Frame                1745
Time (sec)       3.490000
stand_x         28.118267
stand_y       -101.134621
stand_z        157.580765
rim_top_x       24.135523
rim_top_y      267.083313
rim_top_z      280.995087
axel_x          35.105690
axel_y          10.320485
axel_z         286.850586
Name: 1748, dtype: object
Frame                1746
Time (sec)       3.492000
stand_x     

Frame                2496
Time (sec)       4.992000
stand_x         28.114838
stand_y       -101.130180
stand_z        157.576370
rim_top_x       24.130796
rim_top_y      267.087219
rim_top_z      280.988770
axel_x          35.105164
axel_y          10.322200
axel_z         286.855011
Name: 2499, dtype: object
Frame                2497
Time (sec)       4.994000
stand_x         28.115807
stand_y       -101.128670
stand_z        157.575256
rim_top_x       24.133366
rim_top_y      267.085052
rim_top_z      280.995331
axel_x          35.107403
axel_y          10.329896
axel_z         286.848999
Name: 2500, dtype: object
Frame                2498
Time (sec)       4.996000
stand_x         28.111580
stand_y       -101.128601
stand_z        157.576431
rim_top_x       24.134333
rim_top_y      267.085236
rim_top_z      280.995758
axel_x          35.103832
axel_y          10.321874
axel_z         286.853027
Name: 2501, dtype: object
Frame                2499
Time (sec)       4.998000
stand_x     

Frame                3222
Time (sec)       6.444000
stand_x         28.106953
stand_y       -101.070808
stand_z        157.576202
rim_top_x       23.738834
rim_top_y      266.949036
rim_top_z      281.212769
axel_x          35.044743
axel_y          10.229004
axel_z         286.894501
Name: 3225, dtype: object
Frame                3223
Time (sec)       6.446000
stand_x         28.097740
stand_y       -101.080887
stand_z        157.584229
rim_top_x       23.705490
rim_top_y      266.943512
rim_top_z      281.205383
axel_x          35.033783
axel_y          10.222134
axel_z         286.903381
Name: 3226, dtype: object
Frame                3224
Time (sec)       6.448000
stand_x         28.092920
stand_y       -101.085396
stand_z        157.593445
rim_top_x       23.676731
rim_top_y      266.936340
rim_top_z      281.215332
axel_x          35.028088
axel_y          10.216840
axel_z         286.913147
Name: 3227, dtype: object
Frame                3225
Time (sec)       6.450000
stand_x     

Frame                3995
Time (sec)       7.990000
stand_x         28.099468
stand_y       -101.119194
stand_z        157.521667
rim_top_x       23.724802
rim_top_y      266.922638
rim_top_z      281.165314
axel_x          35.032307
axel_y          10.182082
axel_z         286.849121
Name: 3998, dtype: object
Frame                3996
Time (sec)       7.992000
stand_x         28.095757
stand_y       -101.108727
stand_z        157.522827
rim_top_x       23.719322
rim_top_y      266.926453
rim_top_z      281.168152
axel_x          35.026108
axel_y          10.182891
axel_z         286.849518
Name: 3999, dtype: object
Frame                3997
Time (sec)       7.994000
stand_x         28.094719
stand_y       -101.109375
stand_z        157.519730
rim_top_x       23.710449
rim_top_y      266.912476
rim_top_z      281.184570
axel_x          35.018986
axel_y          10.185809
axel_z         286.858246
Name: 4000, dtype: object
Frame                3998
Time (sec)       7.996000
stand_x     

4599

In [ ]:
#Getting all of the y values of each marker and finding the max
firstRow = df.iloc[0]
YVals = firstRow[3::3]

maxY = max(YVals)

#Labeling the columns corresponding to the rim top marker
for index in YVals.index:
    if YVals[index] == maxY:
        # Rim Top Y Column -> rimTopY
        num_index = df.columns.get_loc(index)
        df.rename(columns={df.columns[num_index]: 'rim_top_y'}, inplace=True)
        df.rename(columns={df.columns[num_index-1]: 'rim_top_x'}, inplace=True)
        df.rename(columns={df.columns[num_index+1]: 'rim_top_z'}, inplace=True)
        YVals = YVals.drop(index)

#recalculate max to find axel column
maxY = max(YVals)

for index in YVals.index:
    if YVals[index] == maxY:
        # Axel Top Y Column -> axelY
        num_index = df.columns.get_loc(index)
        df.rename(columns={df.columns[num_index]: 'axel_y'}, inplace=True)
        df.rename(columns={df.columns[num_index-1]: 'axel_x'}, inplace=True)
        df.rename(columns={df.columns[num_index+1]: 'axel_z'}, inplace=True)
        YVals = YVals.drop(index)
        
#Re-labelling the xyz columns of the stand marker
num_index = df.columns.get_loc(YVals.index[0])
df.rename(columns={df.columns[num_index]: 'stand_y'}, inplace=True)
df.rename(columns={df.columns[num_index-1]: 'stand_x'}, inplace=True)
df.rename(columns={df.columns[num_index+1]: 'stand_z'}, inplace=True)


#finds original distance between the stand and rim markers
firstRow = df.iloc[0]

distRimStand0 = {}

distRimStandx = float(firstRow['rim_top_x'])-float(firstRow['stand_x'])
distRimStand0["x"] = distRimStandx

distRimStandy = float(firstRow['rim_top_y'])-float(firstRow['stand_y'])
distRimStand0["y"] = distRimStandy

distRimStandz = float(firstRow['rim_top_z'])-float(firstRow['stand_z'])
distRimStand0["z"] = distRimStandz
